In [ ]:
from keras import Sequential

model = Sequential()

In [ ]:
import csv
import numpy as np
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Embedding, GRU, SimpleRNN, RNN, LSTM, Flatten, Activation
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_fscore_support, accuracy_score
# from baseline import bong
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import time

In [ ]:
def get_data(train_file, test_file, padding='post'):
    list_of_sentences_train = list()
    list_of_hateful_train = list()
    list_of_aggressive_train = list()
    list_of_targeted_train = list()
    max_length = list()

    ### Train file ###
    with open(train_file, encoding="utf8") as cur_train_file:
        reader = csv.reader(cur_train_file, delimiter='\t')
        for row in reader:
            list_of_sentences_train.append(row[1])
            list_of_hateful_train.append(row[2])
            list_of_targeted_train.append(row[3])
            list_of_aggressive_train.append(row[4])

    list_of_sentences_train.pop(0)
    list_of_hateful_train.pop(0)
    list_of_targeted_train.pop(0)
    list_of_aggressive_train.pop(0)

    for x in list_of_sentences_train:
        max_length.append(len(x))
    longest_sent = max(max_length)

    alphabet = dict()
    train_words = []

    for sent in list_of_sentences_train:
        words = []
        for char in sent:
            if char in alphabet:
                words.append(alphabet[char])
            else:
                alphabet[char] = len(alphabet) + 1
                words.append(alphabet[char])

        train_words.append(words)

    train_x = train_words
    #train_y_hateful = list_of_hateful_train
    train_y_hateful = [int(label) for label in list_of_hateful_train]
    train_y_targeted = list_of_targeted_train
    train_y_aggressive = list_of_aggressive_train

    train_x = pad_sequences(train_x, padding=padding, value=0, maxlen=longest_sent, truncating='post')

    ### TEST FIlE ###

    list_of_sentences_test = list()
    list_of_hateful_test = list()
    list_of_aggressive_test = list()
    list_of_targeted_test = list()

    with open(test_file, encoding="utf8") as cur_test_file:
        reader = csv.reader(cur_test_file, delimiter='\t')
        for row in reader:
            list_of_sentences_test.append(row[1])
            list_of_hateful_test.append(row[2])
            list_of_aggressive_test.append(row[3])
            list_of_targeted_test.append(row[4])

        list_of_sentences_test.pop(0)
        list_of_hateful_test.pop(0)
        list_of_aggressive_test.pop(0)
        list_of_targeted_test.pop(0)

        test_words = []
        alphabet_length = len(alphabet)

        for test_sent in list_of_sentences_test:
            words = []
            for char_test in test_sent:
                if char_test in alphabet:
                    words.append(alphabet[char_test])
                else:
                    unk = alphabet_length + 1
                    words.append(unk)

            test_words.append(words)

        test_x = test_words
        #test_y_hateful = list_of_hateful_test
        test_y_hateful = [int(label) for label in list_of_hateful_test]
        test_y_aggressive = list_of_aggressive_test
        test_y_targeted = list_of_targeted_test

        test_x = pad_sequences(test_x, padding=padding, value=0, maxlen=longest_sent, truncating='post')

        print(len(train_x), len(train_y_hateful),len(train_y_aggressive), len(train_y_targeted), len(test_x),
             len(test_y_hateful), len(test_y_aggressive), len(test_y_targeted))

        return train_x, train_y_hateful, train_y_aggressive, train_y_targeted, test_x, test_y_hateful, test_y_aggressive, test_y_targeted, longest_sent

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
from tensorflow.keras.utils import plot_model

train_x, train_y_hateful, train_y_aggressive, train_y_targeted, test_x, test_y_hateful, test_y_aggressive, test_y_targeted, max_sent = get_data(
        train_file='/content/train_en.tsv',
        test_file='/content/dev_en.tsv',
        padding='post')

train_x = np.asarray(train_x)
train_y = np.asarray(train_y_hateful)
train_y = np.reshape(train_y, (-1, 1))
test_x = np.asarray(test_x)
test_y = np.asarray(test_y_hateful)
test_y = np.reshape(test_y, (-1, 1))

recurrent_model = Sequential()
recurrent_model.add(Embedding(input_dim=5000, output_dim=28, input_length=max_sent, mask_zero=False))
recurrent_model.add(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))
recurrent_model.add(Flatten())

recurrent_model.add(Dense(units=1, activation='sigmoid'))
recurrent_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

plot_model(recurrent_model, to_file="/content/embedded_lstm.png", show_shapes=True, show_dtype=False, show_layer_names=False, show_trainable=True, show_layer_activations=True)

# Set callback functions to early stop training and save the best model so far
es = EarlyStopping(monitor='val_loss', patience=5)

# Train neural network
    # recurrent_model.fit(train_x, train_y, validation_split=0.2, epochs=100, callbacks=[es], batch_size=64)
time1 = time.ctime()
recurrent_model.fit(train_x, train_y, callbacks=[es], epochs=50, batch_size=128)
time2 = time.ctime()

9000 9000 9000 9000 1000 1000 1000 1000
Epoch 1/50
71/71 [==============================] - ETA: 0s - loss: 0.6515 - accuracy: 0.6160

71/71 [==============================] - 102s 1s/step - loss: 0.6515 - accuracy: 0.6160
Epoch 2/50
71/71 [==============================] - ETA: 0s - loss: 0.5859 - accuracy: 0.6843

71/71 [==============================] - 90s 1s/step - loss: 0.5859 - accuracy: 0.6843
Epoch 3/50
71/71 [==============================] - ETA: 0s - loss: 0.5744 - accuracy: 0.6902

71/71 [==============================] - 87s 1s/step - loss: 0.5744 - accuracy: 0.6902
Epoch 4/50
71/71 [==============================] - ETA: 0s - loss: 0.5655 - accuracy: 0.7022

71/71 [==============================] - 87s 1s/step - loss: 0.5655 - accuracy: 0.7022
Epoch 5/50
71/71 [==============================] - ETA: 0s - loss: 0.5533 - accuracy: 0.7114

...

71/71 [==============================] - 90s 1s/step - loss: 0.1430 - accuracy: 0.9480
Epoch 47/50
71/71 [==============================] - ETA: 0s - loss: 0.1373 - accuracy: 0.9479

71/71 [==============================] - 87s 1s/step - loss: 0.1373 - accuracy: 0.9479
Epoch 48/50
71/71 [==============================] - ETA: 0s - loss: 0.1237 - accuracy: 0.9573

71/71 [==============================] - 87s 1s/step - loss: 0.1237 - accuracy: 0.9573
Epoch 49/50
71/71 [==============================] - ETA: 0s - loss: 0.1126 - accuracy: 0.9612

71/71 [==============================] - 89s 1s/step - loss: 0.1126 - accuracy: 0.9612
Epoch 50/50
71/71 [==============================] - ETA: 0s - loss: 0.1070 - accuracy: 0.9637

71/71 [==============================] - 89s 1s/step - loss: 0.1070 - accuracy: 0.9637


In [ ]:
recurrent_model.save_weights('/content/embedded_lstm_weights.h5')

y_test_pred_hateful = recurrent_model.predict(test_x)


smaller_than = 0.499
list_of_numbers = list()

for y in y_test_pred_hateful:
        #print(int(y))
    if y <= smaller_than:
        list_of_numbers.append(int("0"))
    else:
        list_of_numbers.append(int("1"))

print(list_of_numbers)

test_y_new = []
for x in test_y:
    x_new = float(x)
    x_new = np.asarray(x_new)
    test_y_new.append(x_new)

accuracy = accuracy_score(test_y_new, y_test_pred_hateful.round())
prec = precision_score(test_y_new, y_test_pred_hateful.round(), average='macro')
rec = recall_score(test_y_new, y_test_pred_hateful.round(), average='macro')
f1 = f1_score(test_y_new, y_test_pred_hateful.round(), average='macro')
#print(y_test_pred_hateful)


print(recurrent_model.summary())
print("Precision:", prec, "\n Recall:", rec, "\n F1-score:", f1, "n Accuracy:", accuracy)
print(f"Start time:{time1} \n End time:{time2}")

32/32 [==============================] - 6s 175ms/step
[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,

In [ ]:
import csv
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Flatten, Dense
from keras.preprocessing.sequence import pad_sequences
import keras

def load_model(max_sent, weights_path):
    # Create the model architecture - this has to match the architecture of the trained model
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=28, input_length=max_sent, mask_zero=False))
    model.add(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))
    model.add(Flatten())
    model.add(Dense(units=1, activation='sigmoid'))

    # Load the trained weights
    model.load_weights(weights_path)

    return model

def get_max_sentence_and_alphabet(train_file):
  list_of_sentences_train = list()
  max_length = list()

    ### Train file ###
  with open(train_file, encoding="utf8") as cur_train_file:
      reader = csv.reader(cur_train_file, delimiter='\t')
      for row in reader:
          list_of_sentences_train.append(row[1])

  list_of_sentences_train.pop(0)

  for x in list_of_sentences_train:
      max_length.append(len(x))
  longest_sent = max(max_length)

  alphabet = dict()

  for sent in list_of_sentences_train:
      for char in sent:
          if char not in alphabet:
            alphabet[char] = len(alphabet) + 1
  return alphabet, longest_sent

def preprocess_input(sentence, alphabet, max_sent):
    # Convert each character in the sentence to its corresponding integer
    sentence_integers = []
    alphabet_length = len(alphabet)
    for char in sentence:
        if char in alphabet:
            sentence_integers.append(alphabet[char])
        else:
            unk = alphabet_length + 1  # For unknown characters
            sentence_integers.append(unk)

    # Pad the sequence to the maximum length used during training
    padded_seq = pad_sequences([sentence_integers], maxlen=max_sent, padding='post', value=0)

    return padded_seq


# Usage example
weights_path = '/content/embedded_lstm_weights.h5'  # Path to your saved weights

alphabet, max_sent = get_max_sentence_and_alphabet("/content/train_en.tsv")
#print(f"maximum length of a sentence = {max_sent}/nAlphabet dictionary : {alphabet}")
model = load_model(max_sent, weights_path)

# Example usage
sentence = input("Enter your sentence: ")

processed_sentence = preprocess_input(sentence, alphabet, max_sent)
prediction = model.predict(processed_sentence)
predicted_label = "Hateful" if prediction[0][0] > 0.5 else "Non-Hateful"
print(f"The sentence is: {predicted_label}")

Enter your sentence: Donald Trump is an asshole
1/1 [==============================] - 0s 322ms/step
The sentence is: Hateful
